In [1]:
%matplotlib inline
import glob

from pandas import DataFrame, read_csv

import pandas as pd

#Versioning
import sys

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

Python version 3.5.1 |Anaconda 2.5.0 (64-bit)| (default, Dec  7 2015, 11:16:01) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Pandas version 0.17.1


In [3]:
search_words = ['non-coding',
                'intergenic',
                'intron',
                'exon',
                'promoter-TSS',
                'TTS',
                '5\' UTR',
                '3\' UTR']

In [4]:
files_ = glob.glob('*.anno')
genes_ = [read_csv(f, sep='\t') for f in files_]

for gene_ in genes_:
    # Fix the PeakID label b/c it had 'junk' in it
    gene_.columns.values[0] = 'PeakID'

genes_data = pd.concat(genes_, keys=files_)

In [5]:
gene_names = genes_data.groupby(level=0)['Gene Name'].value_counts()
gene_names.groupby(level=0).head()

                         Gene Name 
Botcheve_IMR90.anno      CDKN1A        3
                         GDF15         3
                         IRF2BPL       3
                         KLLN          3
                         SUGCT         3
Akdemir_p53_DOX.anno     HTR2C         6
                         LPHN2         6
                         UTRN          6
                         C8orf22       5
                         CDH8          5
Akdemir_p53_Untr.anno    LPHN3         6
                         CDH8          5
                         LOC389906     5
                         WBSCR17       5
                         AC011899.9    4
Younger_Human_ChIP.anno  AUTS2         5
                         CDC42EP3      5
                         PRR26         5
                         BMPER         4
                         CDH8          4
dtype: int64

In [6]:
# Table contains all of the rows with the gene 'CDKN1A'
genes_data[genes_data['Gene Name'].str.contains('CDKN1A', na=False)]

PeakID   Chr     Start       End Strand  \
Botcheve_IMR90.anno     11          36752168  chr6  36751791  36752590      +   
                        97          36753075  chr6  36752686  36753373      +   
                        231         36758405  chr6  36758122  36758684      +   
Akdemir_p53_DOX.anno    37    MACS_peak_3944  chr6  36751522  36756745      +   
                        2586  MACS_peak_3945  chr6  36758106  36759518      +   
Akdemir_p53_Untr.anno   38    MACS_peak_3558  chr6  36751654  36752444      +   
Younger_Human_ChIP.anno 977             1964  chr6  36634693  36635391      +   
                        2301            1965  chr6  36643697  36646241      +   
                        2511            1966  chr6  36649603  36651324      +   

                              Peak Score  Focus Ratio/Region Size  \
Botcheve_IMR90.anno     11        114.00                      NaN   
                        97         30.00                      NaN   
                        231        18.00                      NaN   
Akdemir_p53_DOX.anno    37       3100.00                      NaN   
                        2586      145.35                      NaN   
Akdemir_p53_Untr.anno   38        935.46                      NaN   
Younger_Human_ChIP.anno 977         0.00                      NaN   
                        2301        0.00                      NaN   
                        2511        0.00                      NaN   

                                                        Annotation  \
Botcheve_IMR90.anno     11                promoter-TSS (NM_078467)   
                        97    intron (NM_001220777, intron 1 of 2)   
                        231   intron (NM_001220778, intron 1 of 2)   
Akdemir_p53_DOX.anno    37                promoter-TSS (NM_000389)   
                        2586  intron (NM_001220778, intron 1 of 2)   
Akdemir_p53_Untr.anno   38                promoter-TSS (NM_078467)   
Younger_Human_ChIP.anno 977                             Intergenic   
                        2301  intron (NM_001220777, intron 1 of 2)   
                        2511  intron (NM_001220778, intron 1 of 2)   

                                               Detailed Annotation  \
Botcheve_IMR90.anno     11                                     NaN   
                        97                                     NaN   
                        231                                    NaN   
Akdemir_p53_DOX.anno    37                promoter-TSS (NM_000389)   
                        2586  intron (NM_001220778, intron 1 of 2)   
Akdemir_p53_Untr.anno   38                promoter-TSS (NM_078467)   
Younger_Human_ChIP.anno 977                             Intergenic   
                        2301  intron (NM_001220777, intron 1 of 2)   
                        2511  intron (NM_001220778, intron 1 of 2)   

                              Distance to TSS Nearest PromoterID  Entrez ID  \
Botcheve_IMR90.anno     11                -25       NM_001220777       1026   
                        97                784          NM_078467       1026   
                        231              3938       NM_001220778       1026   
Akdemir_p53_DOX.anno    37               -301          NM_000389       1026   
                        2586             4347       NM_001220778       1026   
Akdemir_p53_Untr.anno   38               -166       NM_001220777       1026   
Younger_Human_ChIP.anno 977             -9195       NM_001220777       1026   
                        2301              702          NM_078467       1026   
                        2511             3976       NM_001220778       1026   

                             Nearest Unigene Nearest Refseq  Nearest Ensembl  \
Botcheve_IMR90.anno     11         Hs.732576   XM_005248787  ENSG00000124762   
                        97         Hs.732576   XM_005248787  ENSG00000124762   
                        231        Hs.732576   XM_005248787  ENSG00000124762   
Akdemir_p53_DOX.anno   

In [7]:
# Takes the list of search_words and gives a table of the hits and files

hits_mask_ = [genes_data.applymap(lambda x, word=word: word in str(x)).any(axis='columns') for word in search_words]
hits_results_ = [genes_data[mask].groupby(level=0).count().iloc[:, 0] for mask in hits_mask_]
search_data = pd.DataFrame(hits_results_, index=search_words); search_data

,Botcheve_IMR90.anno,Akdemir_p53_DOX.anno,Akdemir_p53_Untr.anno,Younger_Human_ChIP.anno
non-coding,9,38,34,10
intergenic,14,155,128,64
intron,233,1898,1453,1145
exon,85,262,308,75
promoter-TSS,176,423,802,72
TTS,8,66,66,31
5' UTR,30,66,105,10
3' UTR,6,44,43,22
